In [1]:
import re
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def logs_from_file(f):
    #print "entering logs_from_file with %s" % f
    log = ""
    for line in f:
        if log == "": # first line of log
            log = line # open new log
        else: # already have open log
            if line[0] == "[": # new line is next log
                yield log # close previous log
                log = line # and start new one
            else: # new line is part of open log
                log = log + line  # append new line of log
    yield log # close log at the end of file

df = pd.DataFrame()
#df = df.append({'name': 'Zed', 'age': 9, 'height': 2}, ignore_index=True)

ms = 0
lastts = datetime.min
with open("q.txt") as f:
    for l in logs_from_file(f):
        # https://regex101.com/r/vX1vW1/3M
        # https://regex101.com/r/eO9hB4/1
        match = re.search("^\[(.+)\]\[(.+)\] \[TE_TRACE\]\: ({.+}) Adding emulation request on Image: '([^']+)', Run: (\d+), Priority: (\w+) \((\d+) requests in queue, (\d+) running emulation VMs", l)
        if match:
            #print "ts %s queue %d VMs %d" % (str(datetime.strptime(match.group(2), '%d %b %H:%M:%S')+relativedelta(years=2020-1900)), int(match.group(7)), int(match.group(8)))
            ts = datetime.strptime(match.group(2), '%d %b %H:%M:%S')+relativedelta(years=2020-1900)
            if ts == lastts:
                ms = ms + 1
            else:
                ms = 0
            df = df.append({'ts': ts+relativedelta(microseconds=ms),
                      'queue': int(match.group(7)), 
                       'vms': int(match.group(8))}, ignore_index=True)
        
            lastts = ts

df=df.set_index(["ts"])         
df.head(5)


,queue,vms
ts,,
2017-12-08 17:37:45.000000,0.0,11.0
2017-12-08 17:37:45.000001,1.0,11.0
2017-12-08 17:37:45.000002,2.0,11.0
2017-12-08 17:37:45.000003,1.0,13.0
2017-12-08 17:37:45.000004,2.0,13.0


In [3]:
import charts
options = dict(height=400, title=dict(text='Chart'))
charts.plot(df, options, height=500, stock=True, show='inline')

Server running in the folder D:\Work at 127.0.0.1:55614
